# Gerekli kütüphaneleri yükleyin


In [ ]:
import pandas as pd
from lifetimes import BetaGeoFitter, GammaGammaFitter

# Veriyi Hazırlama ve Aykırı Değerleri Baskılama


In [ ]:
def prepare_data(file_path):
    df = pd.read_csv(file_path)
    
    # Tarih değişkenlerini date formatına çevir
    date_columns = ["first_order_date", "last_order_date", "last_order_date_online", "last_order_date_offline"]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col])
    
    # Eksik ve 0 değerleri kontrol et
    print("Eksik değer sayısı:")
    print(df.isnull().sum())
    print("0 değer sayısı:")
    print((df == 0).sum())
    
    # Eksik ve 0 değerleri sil
    df = df.dropna()
    df = df[(df[['order_num_total_ever_online', 'order_num_total_ever_offline', 
                 'customer_value_total_ever_offline', 'customer_value_total_ever_online']] != 0).all(axis=1)]
    
    # Aykırı Değerleri Baskılamak için Fonksiyonları Tanımla
    def outlier_thresholds(dataframe, variable):
        quartile1 = dataframe[variable].quantile(0.01)
        quartile3 = dataframe[variable].quantile(0.99)
        interquartile_range = quartile3 - quartile1
        up_limit = quartile3 + 1.5 * interquartile_range
        low_limit = quartile1 - 1.5 * interquartile_range
        return low_limit, up_limit

    def replace_with_thresholds(dataframe, variable):
        low_limit, up_limit = outlier_thresholds(dataframe, variable)
        dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
        dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

    # Belirtilen Değişkenlerin Aykırı Değerlerini Baskıla
    variables = ["order_num_total_ever_online", "order_num_total_ever_offline", 
                 "customer_value_total_ever_offline", "customer_value_total_ever_online"]

    for variable in variables:
        replace_with_thresholds(df, variable)
    
    # Toplam Alışveriş Sayısı ve Harcaması için Yeni Değişkenler Oluştur
    df["total_order"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
    df["total_value"] = df["customer_value_total_ever_online"] + df["customer_value_total_ever_offline"]
    
    return df


# CLTV Veri Yapısının Oluşturulması


In [ ]:
def create_cltv_dataframe(df):
    # Analiz tarihi
    analysis_date = df["last_order_date"].max() + pd.DateOffset(days=2)
    
    # Recency ve Tenure hesaplama (haftalık olarak)
    df["recency_cltv_weekly"] = (df["last_order_date"] - df["first_order_date"]).dt.days / 7
    df["T_weekly"] = (analysis_date - df["first_order_date"]).dt.days / 7
    
    # Frequency ve Monetary hesaplama
    df["frequency"] = df["total_order"]
    df["monetary_cltv_avg"] = df["total_value"] / df["total_order"]
    
    # CLTV dataframe'i oluşturma
    cltv_df = df.loc[:, ["master_id", "recency_cltv_weekly", "T_weekly", "frequency", "monetary_cltv_avg"]]
    cltv_df.columns = ["customer_id", "recency_cltv_weekly", "T_weekly", "frequency", "monetary_cltv_avg"]
    
    # Frequency değerlerinin integer olması gerekiyor
    cltv_df["frequency"] = cltv_df["frequency"].astype(int)
    
    return cltv_df



# BG/NBD ve Gamma-Gamma Modellerinin Kurulması, CLTV'nin Hesaplanması

In [ ]:
def fit_bgf_ggf_models(cltv_df):
    # BG/NBD modelini fit et
    bgf = BetaGeoFitter(penalizer_coef=0.001)
    bgf.fit(cltv_df['frequency'], cltv_df['recency_cltv_weekly'], cltv_df['T_weekly'])
    
    # 3 ay içerisinde beklenen satın almalar
    cltv_df['exp_sales_3_month'] = bgf.predict(3*4, cltv_df['frequency'], cltv_df['recency_cltv_weekly'], cltv_df['T_weekly'])
    
    # 6 ay içerisinde beklenen satın almalar
    cltv_df['exp_sales_6_month'] = bgf.predict(6*4, cltv_df['frequency'], cltv_df['recency_cltv_weekly'], cltv_df['T_weekly'])
    
    # Gamma-Gamma modelini fit et
    ggf = GammaGammaFitter(penalizer_coef=0.01)
    ggf.fit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])
    
    # Tahmin edilen ortalama değeri dataframe'e ekle
    cltv_df['exp_average_value'] = ggf.conditional_expected_average_profit(cltv_df['frequency'], cltv_df['monetary_cltv_avg'])
    
    # 6 aylık CLTV hesapla
    cltv_df['cltv'] = ggf.customer_lifetime_value(
        bgf, 
        cltv_df['frequency'], 
        cltv_df['recency_cltv_weekly'], 
        cltv_df['T_weekly'], 
        cltv_df['monetary_cltv_avg'], 
        time=6,  # 6 aylık
        freq='W',  # Haftalık
        discount_rate=0.01  # İskonto oranı
    )
    
    return cltv_df

# CLTV'ye Göre Segmentlerin Oluşturulması


In [ ]:
def create_segments(cltv_df):
    # Müşterileri CLTV'ye göre segmentlere ayır
    cltv_df['cltv_segment'] = pd.qcut(cltv_df['cltv'], 4, labels=["D", "C", "B", "A"])
    return cltv_df

def action_recommendations():
    recommendations = {
        "Group A": [
            "Sadakat Programları: En değerli müşterilerinize özel sadakat programları sunun. Özel indirimler, erken erişim fırsatları ve ücretsiz kargo gibi avantajlar sağlayarak sadakati artırın.",
            "Kişiselleştirilmiş Pazarlama: Müşterilerin önceki alışveriş alışkanlıklarına dayanarak kişiselleştirilmiş pazarlama kampanyaları oluşturun. Onlara özel ürün önerileri ve kampanyalar sunarak müşteri memnuniyetini artırın."
        ],
        "Group B": [
            "Çapraz Satış Stratejileri: Bu gruptaki müşterilere ilgili ürünleri sunarak çapraz satış fırsatlarını değerlendirin. Örneğin, bir ayakkabı satın alan müşteriye uyumlu çorap veya bakım ürünleri önerin.",
            "Müşteri Geri Bildirimi: Müşteri geri bildirimlerini toplamak ve değerlendirmek için anketler düzenleyin. Müşteri memnuniyetini artırmak için geri bildirimlere dayalı iyileştirmeler yapın."
        ]
    }
    return recommendations

# Tüm Süreci Fonksiyonlaştırma


In [ ]:
def main(file_path):
    df = prepare_data(file_path)
    cltv_df = create_cltv_dataframe(df)
    cltv_df = fit_bgf_ggf_models(cltv_df)
    cltv_df = create_segments(cltv_df)
    
    recommendations = action_recommendations()
    
    # CLTV değeri en yüksek 20 kişiyi gözlemle
    top_20_cltv = cltv_df.sort_values(by='cltv', ascending=False).head(20)
    
    return cltv_df, top_20_cltv, recommendations

# Ana fonksiyonu çağırma


In [ ]:
file_path = '/https://github.com/sahindemirbas/FLO_CLTV_Prediction/blob/45b2fcd206d70f8d1caa8b7772229eb2c7a89b4f/flo_data_20k.csv'
cltv_df, top_20_cltv, recommendations = main(file_path)

# Sonuçları görüntüle


In [ ]:
print(cltv_df.head())
print(top_20_cltv)
print(recommendations)